In [2]:
import numpy as np
import numpy.linalg
import scipy.sparse
import scipy.sparse.linalg
from PIL import Image
from numba import njit
# import matplotlib.pyplot as plt

def main():
    # configure paths here
    image_path  = "FBAMATING_PunyaGambar/87ZMG.png"
    trimap_path = "FBAMATING_PunyaGambar/trimaps/87ZMG.png"
    alpha_path  = "cat2.png"
    cutout_path = "cat2_cutout.png"

    # load and convert to [0, 1] range
    image  = np.array(Image.open( image_path).convert("RGB"))/255.0
    trimap = np.array(Image.open(trimap_path).convert(  "L"))/255.0

    # make matting laplacian
    i,j,v = closed_form_laplacian(image)
    h,w = trimap.shape
    L = scipy.sparse.csr_matrix((v, (i, j)), shape=(w*h, w*h))

    # build linear system
    A, b = make_system(L, trimap)

    # solve sparse linear system
    print("solving linear system...")
    alpha = scipy.sparse.linalg.spsolve(A, b).reshape(h, w)

    # stack rgb and alpha
    cutout = np.concatenate([image, alpha[:, :, np.newaxis]], axis=2)

    # clip and convert to uint8 for PIL
    cutout = np.clip(cutout*255, 0, 255).astype(np.uint8)
    alpha  = np.clip( alpha*255, 0, 255).astype(np.uint8)

    # save and show
    Image.fromarray(alpha ).save( alpha_path)
    Image.fromarray(cutout).save(cutout_path)
    Image.fromarray(alpha ).show()
    Image.fromarray(cutout).show()


@njit
def closed_form_laplacian(image, epsilon=1e-7, r=1):
    h,w = image.shape[:2]
    window_area = (2*r + 1)**2
    n_vals = (w - 2*r)*(h - 2*r)*window_area**2
    k = 0
    # data for matting laplacian in coordinate form
    i = np.empty(n_vals, dtype=np.int32)
    j = np.empty(n_vals, dtype=np.int32)
    v = np.empty(n_vals, dtype=np.float64)

    # for each pixel of image
    for y in range(r, h - r):
        for x in range(r, w - r):

            # gather neighbors of current pixel in 3x3 window
            n = image[y-r:y+r+1, x-r:x+r+1]
            u = np.zeros(3)
            for p in range(3):
                u[p] = n[:, :, p].mean()
            c = n - u

            # calculate covariance matrix over color channels
            cov = np.zeros((3, 3))
            for p in range(3):
                for q in range(3):
                    cov[p, q] = np.mean(c[:, :, p]*c[:, :, q])

            # calculate inverse covariance of window
            inv_cov = np.linalg.inv(cov + epsilon/window_area * np.eye(3))

            # for each pair ((xi, yi), (xj, yj)) in a 3x3 window
            for dyi in range(2*r + 1):
                for dxi in range(2*r + 1):
                    for dyj in range(2*r + 1):
                        for dxj in range(2*r + 1):
                            i[k] = (x + dxi - r) + (y + dyi - r)*w
                            j[k] = (x + dxj - r) + (y + dyj - r)*w
                            temp = c[dyi, dxi].dot(inv_cov).dot(c[dyj, dxj])
                            v[k] = (1.0 if (i[k] == j[k]) else 0.0) - (1 + temp)/window_area
                            k += 1
        print("generating matting laplacian", y - r + 1, "/", h - 2*r)

    return i, j, v

def make_system(L, trimap, constraint_factor=100.0):
    # split trimap into foreground, background, known and unknown masks
    is_fg = (trimap > 0.9).flatten()
    is_bg = (trimap < 0.1).flatten()
    is_known = is_fg | is_bg
    is_unknown = ~is_known

    # diagonal matrix to constrain known alpha values
    d = is_known.astype(np.float64)
    D = scipy.sparse.diags(d)

    # combine constraints and graph laplacian
    A = constraint_factor*D + L
    # constrained values of known alpha values
    b = constraint_factor*is_fg.astype(np.float64)

    return A, b

if __name__ == "__main__":
    main()

generating matting laplacian 1 / 510
generating matting laplacian 2 / 510
generating matting laplacian 3 / 510
generating matting laplacian 4 / 510
generating matting laplacian 5 / 510
generating matting laplacian 6 / 510
generating matting laplacian 7 / 510
generating matting laplacian 8 / 510
generating matting laplacian 9 / 510
generating matting laplacian 10 / 510
generating matting laplacian 11 / 510
generating matting laplacian 12 / 510
generating matting laplacian 13 / 510
generating matting laplacian 14 / 510
generating matting laplacian 15 / 510
generating matting laplacian 16 / 510
generating matting laplacian 17 / 510
generating matting laplacian 18 / 510
generating matting laplacian 19 / 510
generating matting laplacian 20 / 510
generating matting laplacian 21 / 510
generating matting laplacian 22 / 510
generating matting laplacian 23 / 510
generating matting laplacian 24 / 510
generating matting laplacian 25 / 510
generating matting laplacian 26 / 510
generating matting la

In [17]:
from pymatting import cutout

image_path  = "FBAMATING_PunyaGambar/every-type-of-bike-c052991.jpg"
trimap_path = "FBAMATING_PunyaGambar/trimaps/every-type-of-bike-c052991.png"
alpha_path  = "bikebike.png"
cutout_path = "bikebike_cutout.png"

cutout(
   # input image path
   image_path,
   # input trimap path
   trimap_path,
   # output cutout path
   "bikebike_cutout.png")

In [18]:
from pymatting import *
import numpy as np

scale = 1.0

image = load_image(image_path, "RGB", scale, "box")
trimap = load_image(trimap_path, "GRAY", scale, "nearest")

# estimate alpha from image and trimap
alpha = estimate_alpha_cf(image, trimap)

# make gray background
background = np.zeros(image.shape)
background[:, :] = [0.5, 0.5, 0.5]

# estimate foreground from image and alpha
foreground = estimate_foreground_ml(image, alpha)

# blend foreground with background and alpha, less color bleeding
new_image = blend(foreground, background, alpha)

# save results in a grid
images = [image, trimap, alpha, new_image]
grid = make_grid(images)
save_image("bikebike_grid.png", grid)

# save cutout
cutout = stack_images(foreground, alpha)
save_image("bikebike_cutout2.png", cutout)

# just blending the image with alpha results in color bleeding
color_bleeding = blend(image, background, alpha)
grid = make_grid([color_bleeding, new_image])
save_image("bikebike.png", grid)

In [7]:
from pymatting import *
import numpy as np
import scipy.sparse

scale = 1.0

image = load_image(image_path, size=scale, resample="box")
trimap = load_image(trimap_path, mode="GRAY", size=scale, resample="nearest")

# height and width of trimap
h, w = trimap.shape[:2]

# calculate laplacian matrix
L = cf_laplacian(image)

# decompose trimap
is_fg, is_bg, is_known, is_unknown = trimap_split(trimap)

# constraint weight
lambda_value = 100.0

# build constraint pixel selection matrix
c = lambda_value * is_known
C = scipy.sparse.diags(c)

# build constraint value vector
b = lambda_value * is_fg

# build linear system
A = L + C

# build ichol preconditioner for faster convergence
A = A.tocsr()
A.sum_duplicates()
M = ichol(A)

# solve linear system with conjugate gradient descent
x = cg(A, b, M=M)

# clip and reshape result vector
alpha = np.clip(x, 0.0, 1.0).reshape(h, w)

save_image("lemur_alpha.png", alpha)

In [ ]:
import cv2

alpha_image = 'lemur_alpha.png'
real_image = image_path



In [13]:
# import PIL module
from PIL import Image
# Load the images
myBackgroundImage = Image.open("FBAMATING_PunyaGambar/bg2.jpg")
myForegroundImage = Image.open("bikebike_cutout.png")
# Create a new image with the same size as the background image
myMerged_image = Image.new("RGBA", myBackgroundImage.size)
# Paste the background image onto the new image
myMerged_image.paste(myBackgroundImage, (0, 0))
# Create a mask from the alpha channel of the foreground image
_, _, _, mask = myForegroundImage.split()
# Paste the foreground image onto the new image with the mask
myMerged_image.paste(myForegroundImage, (0, 0), mask)
# Saving the merged image to a file
myMerged_image.save("yourmergedbike.jpg", 'jpeg')

OSError: cannot write mode RGBA as JPEG

In [16]:
from PIL import Image as img

# Open images
cat_image = img.open("bikebike_cutout.png")
background = img.open("FBAMATING_PunyaGambar/bg2.jpg")

width, height = cat_image.size
background = background.resize((width * 2, height * 2), img.LANCZOS)

paste_position = (background.width // 2 - width // 2, background.height // 2 - height // 2)

background.paste(cat_image, paste_position, mask=cat_image)

background.show()